# Agentic Memory with Oracle AI Vector Search
**Building semantic memory for conversation agents**

## Background

This notebook explores semantic memory for AI agents using Oracle Autonomous Database.

Context:
Last year, I worked on a conversation agent for a loan optimization system. The agent would call borrowers, have conversations in multiple languages, and record summaries. The challenge was finding relevant information from past conversations - keyword search didn't work well with conversational data.

This demo explores how semantic memory with vector search could have solved that problem.

## What We're Building

A memory system that allows AI agents to:
- Store conversations as semantic embeddings
- Search by meaning, not keywords
- Retrieve relevant context for better responses

**Why Oracle?**
- Native VECTOR type (no extensions needed)
- Hybrid SQL + vector queries in one database
- Local embeddings (data doesn't leave your server)
- ACID transactions for data integrity

## Table of Contents
1. Setup & Dependencies
2. Database Connection
3. Memory System Implementation
4. Storing Memories
5. Semantic Search Examples
6. RAG Pattern
7. Production Considerations

## 1. Setup & Dependencies

We're using `sentence-transformers` for embeddings because:
- Runs locally (important for data protection)
- No API costs
- Fast enough for real-time use (~50ms per text)

Model: `all-MiniLM-L6-v2` (384 dimensions)

In [1]:
# Install sentence-transformers
!pip install sentence-transformers --break-system-packages
# !pip install oracledb

In [2]:
# Import required libraries
import oracledb
import numpy as np
from typing import List, Dict
from sentence_transformers import SentenceTransformer
import re
from pathlib import Path
import os
import socket

## 2. Database Connection

Prerequisites:
- Oracle Autonomous Database (version 23.26.1.1.0)
- Oracle Instant Client installed
- Wallet files configured
- Connection established from previous cells

In [ ]:
# Path to the folder of the wallet
# Replace with your folder path for your own use case
WALLET_DIR = r"C:\Users\DELL\Downloads\oracle_wallet\Wallet_AGENTMEM"

# Database credentials
USER = "ADMIN"
PASSWORD = "DATABASE_PASSWORD" # Please update the database password for your Oracle DB.

# Service name from tnsnames.ora
DSN = "agentmem_low"

# Set TNS_ADMIN environment variable
os.environ["TNS_ADMIN"] = WALLET_DIR

In [4]:
# Verify all required wallet files exist
required_files = ["tnsnames.ora", "sqlnet.ora", "cwallet.sso", "ewallet.pem"]

print("Checking wallet files:")
all_exist = True
for f in required_files:
    p = Path(WALLET_DIR) / f
    exists = p.exists()
    size = p.stat().st_size if exists else 0
    print(f"{f}: {'Yes' if exists else 'No'} ({size} bytes)")
    if not exists:
        all_exist = False

if not all_exist:
    print("\nWarning: Some wallet files are missing!")
else:
    print("\nAll wallet files present")

Checking wallet files:
tnsnames.ora: Yes (1285 bytes)
sqlnet.ora: Yes (155 bytes)
cwallet.sso: Yes (6349 bytes)
ewallet.pem: Yes (7004 bytes)

All wallet files present


In [5]:
# Initialize thick mode
# Download Oracle Instant Client from: https://www.oracle.com/database/technologies/instant-client/downloads.html
# Then specify the path to the instantclient folder

INSTANT_CLIENT_DIR = r"C:\oracle\instantclient_21_13\instantclient_23_0"

try:
    oracledb.init_oracle_client(lib_dir=INSTANT_CLIENT_DIR)
    print("Oracle thick mode initialized successfully.")
    print(f"Client version: {oracledb.clientversion()}")
except Exception as e:
    print(f"Failed to initialize thick mode: {e}")

Oracle thick mode initialized successfully.
Client version: (23, 26, 0, 0, 0)


In [6]:
# Connect using DSN
try:
    connection = oracledb.connect(
        user=USER,
        password=PASSWORD,
        dsn=DSN,
        config_dir=WALLET_DIR,
        wallet_location=WALLET_DIR,
    )
    print("Successfully connected to Oracle Autonomous Database.")    
    cursor = connection.cursor()
    cursor.execute("SELECT banner FROM v$version WHERE banner LIKE 'Oracle%'")
    version = cursor.fetchone()[0]
    print(f"Version: {version}")
    print(f"Database version: {connection.version}")
    cursor.close()
    
except oracledb.DatabaseError as e:
    error_obj, = e.args
    print(f"Database error: {error_obj.message}")
    print(f"Error code: {error_obj.code}")
except Exception as e:
    print(f"Connection failed: {e}")

Successfully connected to Oracle Autonomous Database.
Version: Oracle AI Database 26ai Enterprise Edition Release 23.26.1.1.0 - for Oracle Cloud and Engineered Systems
Database version: 23.26.1.1.0


In [7]:
print("Loading embedding model.")
EMBEDDING_MODEL = SentenceTransformer('all-MiniLM-L6-v2')
EMBEDDING_DIM = 384
print("Model loaded successfully.")
print(f"Model: all-MiniLM-L6-v2")
print(f"Embedding dimension: {EMBEDDING_DIM}")

Loading embedding model.
Model loaded successfully.
Model: all-MiniLM-L6-v2
Embedding dimension: 384


## 3. Memory System Implementation

The `AgenticMemory` class handles:

### Database Schema

```sql
CREATE TABLE memories (
    id NUMBER PRIMARY KEY,
    text CLOB,                      -- Conversation summary or context
    embedding VECTOR(384, FLOAT32), -- Semantic representation
    memory_type VARCHAR2(50),       -- Category: promise, issue, preference, etc.
    created_at TIMESTAMP            -- For temporal queries
);
```

Key feature: Oracle's native `VECTOR(384, FLOAT32)` type- no extensions needed.

### Operations

1. setup(): Creates the schema.
2. store(): Generates embedding and saves memory.
3. search(): Finds similar memories using cosine similarity.

In [8]:
class AgenticMemory:
    # Semantic memory system for conversation agents.
   
    def __init__(self, connection):
        #Initialize with active Oracle connection.
        self.connection = connection
        self.cursor = connection.cursor()
        
    def setup(self):
        # Create memories table with VECTOR column.
        # Drops existing table if present.
        # In production, use ALTER TABLE or migrations.
        print("Setting up memory table.")
        
        # Drop existing table
        try:
            self.cursor.execute("DROP TABLE memories CASCADE CONSTRAINTS")
            print("Dropped existing table.")
        except:
            pass
        
        # Create table with VECTOR column
        self.cursor.execute(f"""
            CREATE TABLE memories (
                id NUMBER GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
                text CLOB NOT NULL,
                embedding VECTOR({EMBEDDING_DIM}, FLOAT32),
                memory_type VARCHAR2(50),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        self.connection.commit()
        print("Created memories table.")
        print(f"VECTOR column: {EMBEDDING_DIM} dimensions, FLOAT32")
        print()
    
    def store(self, text: str, memory_type: str = "general"):
        # Store a memory with its semantic embedding.
        # Args:
        # text: Memory content (conversation summary, context, etc.)
        # memory_type: Category (promise, issue, preference, context, etc.)         
        # Uses local embedding model, no data leaves your server.
        # Important for data protection in regulated industries.

        # Generate embedding locally
        embedding = EMBEDDING_MODEL.encode(text)
        
        # Convert to string format for Oracle TO_VECTOR function
        # (Required as Oracle Instant Client is in thick mode.
        embedding_list = embedding.tolist()
        array_str = str(embedding_list)
        
        # Store in database
        self.cursor.execute(f"""
            INSERT INTO memories (text, embedding, memory_type)
            VALUES (:1, TO_VECTOR(:2), :3)
        """, [text, array_str, memory_type])
        
        self.connection.commit()
        print(f"Stored: {text}")
    
    def search(self, query: str, top_k: int = 5) -> List[Dict]:
        
        # Search memories using semantic similarity.
        # Uses Oracle's VECTOR_DISTANCE function with COSINE metric.
        # Lower distance = more similar, so we return (1 - distance) as similarity score.
        # Args:
        # query: Search query (natural language)
        # top_k: Number of results to return  
        # Returns:
        # List of dicts with text, type, and similarity score (0-1)
        try:
            # Generate embedding for query
            query_embedding = EMBEDDING_MODEL.encode(query)
            query_list = query_embedding.tolist()
            query_str = str(query_list)
            
            # Search using vector similarity
            self.cursor.execute(f"""
                SELECT 
                    text,
                    memory_type,
                    (1 - VECTOR_DISTANCE(embedding, TO_VECTOR(:1), COSINE)) as similarity
                FROM memories
                ORDER BY similarity DESC
                FETCH FIRST :2 ROWS ONLY
            """, [query_str, top_k])
            
            # Format results
            results = []
            for row in self.cursor:
                results.append({
                    'text': row[0],
                    'type': row[1],
                    'similarity': float(row[2])
                })
            
            return results

        except oracledb.DatabaseError as e:
            error_obj, = e.args
            print(f"Database error during search: {error_obj.message}")
            print(f"Query that failed: {query}")
            return []  # Return empty list instead of crashing
            
        except Exception as e:
            print(f"Unexpected error during search: {e}")
            return []  # Fallback

print("Agentic Memory class defined.")

Agentic Memory class defined.


### Initialize Memory System

In [9]:
# Create instance and setup schema
memory = AgenticMemory(connection)
memory.setup()

Setting up memory table.
Dropped existing table.
Created memories table.
VECTOR column: 384 dimensions, FLOAT32



## 4. Storing Memories

For this demo, we'll use a customer support scenario. 
In the loan system I worked on, these would be conversation summaries like:
- "Borrower promised payment by 15th March"
- "Mentioned medical emergency as reason for delay"
- "Tone was cooperative and understanding"

### Memory Types

Categorizing memories helps with filtering:
- `preference`: User preferences and settings
- `issue`: Problems or complaints
- `inquiry`: Questions asked
- `resolution`: Solutions provided
- `context`: Background information
- `profile`: User profile data

In a loan system, we also have:
- `repayment_promise`
- `interaction_tone`
- `risk_indicator`

In [10]:
print("Storing example memories.\n")

# Store different types of memories

memory.store("Borrower reported app crash while uploading income proof", "issue")
memory.store("Borrower raised complaint about duplicate late fee", "issue")
memory.store("Borrower reported failed auto-debit for February EMI", "issue")

memory.store("Borrower asked about data privacy for bank statements", "inquiry")
memory.store("Borrower asked about foreclosure charges", "inquiry")
memory.store("Borrower asked if EMI date can be shifted", "inquiry")
memory.store("Borrower asked about impact of late payment on credit score", "inquiry")

memory.store("Auto-debit issue resolved after bank mandate update", "resolution")
memory.store("Late fee reversed after manual review", "resolution")
memory.store("Borrower followed through on promised repayment within agreed date", "resolution")

memory.store("Borrower has held a personal loan account since 2021", "profile")
memory.store("Borrower is in GMT timezone", "profile")

memory.store("Borrower income is salaried, paid monthly", "context")
memory.store("Borrower works in healthcare sector", "context")
memory.store("Borrower is employed full-time at a private hospital", "context")
memory.store("Borrower recently relocated to a new city", "context")

memory.store("Borrower prefers EMI reminders 3 days before due date", "preference")
memory.store("Borrower prefers communication during evening hours", "preference")
memory.store("Borrower opts out of promotional loan offers", "preference")

memory.store("Two EMIs paid late in last 6 months", "risk_indicator")
memory.store("Two late payments in last 90 days", "risk_indicator")
memory.store("Borrower frequently contacts support close to due date", "risk_indicator")
memory.store("Temporary income disruption mentioned during call", "risk_indicator")
memory.store("Missed auto-debit twice despite sufficient balance", "risk_indicator")

memory.store("Borrower promised to pay EMI by 15 March", "repayment_promise")
memory.store("Borrower requested one-time payment extension due to medical emergency", "repayment_promise")

memory.store("Borrower was calm and cooperative during call", "interaction_tone")
memory.store("Borrower was polite and appreciative during support chat", "interaction_tone")
memory.store("Borrower sounded stressed while discussing missed payment", "interaction_tone")
memory.store("Borrower became frustrated after multiple follow-ups", "interaction_tone")

print("\nAll memories stored with embeddings.")
print("Each memory is now searchable by semantic meaning.")

Storing example memories.

Stored: Borrower reported app crash while uploading income proof
Stored: Borrower raised complaint about duplicate late fee
Stored: Borrower reported failed auto-debit for February EMI
Stored: Borrower asked about data privacy for bank statements
Stored: Borrower asked about foreclosure charges
Stored: Borrower asked if EMI date can be shifted
Stored: Borrower asked about impact of late payment on credit score
Stored: Auto-debit issue resolved after bank mandate update
Stored: Late fee reversed after manual review
Stored: Borrower followed through on promised repayment within agreed date
Stored: Borrower has held a personal loan account since 2021
Stored: Borrower is in GMT timezone
Stored: Borrower income is salaried, paid monthly
Stored: Borrower works in healthcare sector
Stored: Borrower is employed full-time at a private hospital
Stored: Borrower recently relocated to a new city
Stored: Borrower prefers EMI reminders 3 days before due date
Stored: Borrow

## 5. Semantic Search Examples

Now we'll demonstrate semantic search - finding memories by meaning, not keywords.

What makes this powerful:
- Query: "notification preferences", Finds: "email notifications over SMS"
- Query: "technical problems", Finds: "slow upload speeds" + "fixed by updating"
- Different words, same meaning

### Example 1: Finding User Preferences

In [11]:
print("Semantic Search 1: User Preferences")

query = "What are the user's notification preferences?"
print(f"Query: '{query}'")

results = memory.search(query, top_k=3)

for i, result in enumerate(results, 1):
    print(f"{i}. [Similarity: {result['similarity']:.3f}] {result['text']}")
    print(f"Type: {result['type']}")
    print()

Semantic Search 1: User Preferences
Query: 'What are the user's notification preferences?'
1. [Similarity: 0.228] Borrower prefers EMI reminders 3 days before due date
Type: preference

2. [Similarity: 0.167] Borrower was polite and appreciative during support chat
Type: interaction_tone

3. [Similarity: 0.154] Borrower prefers communication during evening hours
Type: preference



### Example 2: Finding Technical Issues

In [12]:
print("Semantic Search 2: Technical Issues")

query = "Are there any technical problems?"
print(f"Query: '{query}'")

results = memory.search(query, top_k=3)

for i, result in enumerate(results, 1):
    print(f"{i}. [Similarity: {result['similarity']:.3f}] {result['text']}")
    print(f"Type: {result['type']}")
    print()

Semantic Search 2: Technical Issues
Query: 'Are there any technical problems?'
1. [Similarity: 0.208] Temporary income disruption mentioned during call
Type: risk_indicator

2. [Similarity: 0.162] Borrower reported app crash while uploading income proof
Type: issue

3. [Similarity: 0.120] Borrower asked about data privacy for bank statements
Type: inquiry



### Example 3: Finding Compliance Information

In [13]:
print("Semantic Search 3: Compliance Requirements")

query = "Tell me about compliance requirements"
print(f"Query: '{query}'")

results = memory.search(query, top_k=3)

for i, result in enumerate(results, 1):
    print(f"{i}. [Similarity: {result['similarity']:.3f}] {result['text']}")
    print(f"Type: {result['type']}")
    print()

Semantic Search 3: Compliance Requirements
Query: 'Tell me about compliance requirements'
1. [Similarity: 0.210] Borrower asked about data privacy for bank statements
Type: inquiry

2. [Similarity: 0.165] Borrower is employed full-time at a private hospital
Type: context

3. [Similarity: 0.146] Borrower works in healthcare sector
Type: context



### View All Stored Memories

In [14]:
#See all stored memories
cursor = connection.cursor()
cursor.execute(""" SELECT id, text, memory_type, TO_CHAR(created_at, 'YYYY-MM-DD HH24:MI:SS') as created FROM memories ORDER BY created_at""")

print("All Stored Memories:")

for row in cursor:
    print(f"ID {row[0]}: [{row[2]}] {row[1]}")
    print(f"  Created: {row[3]}")
    print()

cursor.close()

All Stored Memories:
ID 1: [issue] Borrower reported app crash while uploading income proof
  Created: 2026-02-09 02:41:12

ID 2: [issue] Borrower raised complaint about duplicate late fee
  Created: 2026-02-09 02:41:12

ID 3: [issue] Borrower reported failed auto-debit for February EMI
  Created: 2026-02-09 02:41:12

ID 4: [inquiry] Borrower asked about data privacy for bank statements
  Created: 2026-02-09 02:41:12

ID 5: [inquiry] Borrower asked about foreclosure charges
  Created: 2026-02-09 02:41:12

ID 6: [inquiry] Borrower asked if EMI date can be shifted
  Created: 2026-02-09 02:41:12

ID 7: [inquiry] Borrower asked about impact of late payment on credit score
  Created: 2026-02-09 02:41:12

ID 8: [resolution] Auto-debit issue resolved after bank mandate update
  Created: 2026-02-09 02:41:12

ID 9: [resolution] Late fee reversed after manual review
  Created: 2026-02-09 02:41:12

ID 10: [resolution] Borrower followed through on promised repayment within agreed date
  Created: 2

### Memory Statistics

In [15]:
# Get some statistics
cursor = connection.cursor()

cursor.execute("SELECT COUNT(*) FROM memories")
total = cursor.fetchone()[0]

cursor.execute(""" SELECT memory_type, COUNT(*) 
    FROM memories 
    GROUP BY memory_type
    ORDER BY COUNT(*) DESC
""")

print(f"Memory Statistics:")
print(f"Total memories: {total}")
print("\nBreakdown by type:")
for row in cursor:
    print(f"{row[0]}: {row[1]}")

cursor.close()

Memory Statistics:
Total memories: 30

Breakdown by type:
risk_indicator: 5
interaction_tone: 4
inquiry: 4
context: 4
resolution: 3
issue: 3
preference: 3
repayment_promise: 2
profile: 2


## 6. RAG Pattern: Retrieval-Augmented Generation

This is how we use agentic memory in a production system.

The RAG Pattern:
1. User asks a question.
2. Search relevant memories (semantic search)
3. Build LLM prompt with retrieved context
4. LLM generates informed, context-aware response

Real-world application (loan system example):

1. Agent about to call borrower
2. Search: "previous conversations with this borrower"
3. Retrieve: "Promised payment next week", "Mentioned medical expenses"
4. Agent starts call with context: "Hi, following up on your payment...I see you mentioned medical expenses last time. How are things now?"

The borrower doesn't have to repeat themselves. Better experience.

In [16]:
def generate_rag_response(user_query: str) -> str:
    # Demonstrate complete RAG pattern.
    # In production, this would call an actual LLM API:
    # - Anthropic Claude API
    # - OpenAI GPT API  
    # - Local models via Ollama
    # For this demo, we simulate the LLM response to show the pattern.

    print(f"User Query: {user_query}\n")
    
    # Retrieve relevant memories
    print("Step 1: Retrieving relevant context.")
    
    results = memory.search(user_query, top_k=3)
    
    for i, result in enumerate(results, 1):
        print(f"{i}. [Similarity: {result['similarity']:.3f}] {result['text']}")
    
    # Build LLM prompt with context
    print("\nStep 2: Building LLM prompt with context.")
    
    context = "\n".join([f"- {r['text']}" for r in results])
    
    # This is what you'd send to Claude/GPT-4/any other
    prompt = f"""You are a helpful customer support agent. Based on the following context from previous interactions:{context} User question: {user_query} Provide a helpful, personalized response."""
    
    print(prompt)
    
    # LLM generates response (simulated)
    print("\nStep 3: LLM generates context-aware response.")
    
    # In production:
    # response = anthropic.messages.create(model="claude-sonnet-4", ...)
    # or
    # response = openai.chat.completions.create(model="gpt-4", ...)
    
    simulated_response = """Based on your preferences, I can see that you prefer email notifications. I'll make sure all important updates are sent to your email address instead of SMS. Is there anything specific you'd like to be notified about?"""
    
    print(f"Agent Response:\n{simulated_response}")
    
    return simulated_response

print("RAG function defined.")

RAG function defined.


### Run RAG Demo

In [17]:
print("RAG Pattern")
print("Retrieval-Augmented Generation with Agentic Memory")
print()

user_query = "How should I contact the user about updates?"
response = generate_rag_response(user_query)

print("\nRAG Pattern complete.")

RAG Pattern
Retrieval-Augmented Generation with Agentic Memory

User Query: How should I contact the user about updates?

Step 1: Retrieving relevant context.
1. [Similarity: 0.269] Borrower frequently contacts support close to due date
2. [Similarity: 0.191] Borrower prefers EMI reminders 3 days before due date
3. [Similarity: 0.185] Temporary income disruption mentioned during call

Step 2: Building LLM prompt with context.
You are a helpful customer support agent. Based on the following context from previous interactions:- Borrower frequently contacts support close to due date
- Borrower prefers EMI reminders 3 days before due date
- Temporary income disruption mentioned during call User question: How should I contact the user about updates? Provide a helpful, personalized response.

Step 3: LLM generates context-aware response.
Agent Response:
Based on your preferences, I can see that you prefer email notifications. I'll make sure all important updates are sent to your email addres

<a id='production'></a>
## 7. Production Considerations

### What To Change for Production

This demo is functional but simplified. For a production system (like the loan conversation agent), we would add:

#### 1. User/Borrower Isolation

```sql
ALTER TABLE memories ADD borrower_id VARCHAR2(100);
CREATE INDEX idx_borrower_id ON memories(borrower_id);

-- Search only this borrower's memories
WHERE borrower_id = :id
AND VECTOR_DISTANCE(embedding, query_vector, COSINE) < 0.3
```

#### 2. Hybrid Queries

Combine semantic search with business logic:

```sql
-- Find similar high-risk cases in a specific region
WHERE risk_score > 0.7
AND geography = 'Maharashtra'
AND created_at > CURRENT_TIMESTAMP - INTERVAL '30' DAY
AND VECTOR_DISTANCE(embedding, query_vector, COSINE) < 0.3
ORDER BY (1 - VECTOR_DISTANCE(embedding, query_vector, COSINE)) DESC
```

#### 3. Vector Index for Scale

```sql
CREATE VECTOR INDEX memory_idx ON memories(embedding)
ORGANIZATION INMEMORY NEIGHBOR GRAPH
WITH TARGET ACCURACY 95;
```

Improves search speed at scale (thousands/millions of vectors).

#### 4. Memory Management

```python
# Consolidate similar memories periodically
# Clean old memories
# Implement importance scoring
```

#### 5. Multi-language Support

For systems handling multiple languages (like the loan agent did):

```python
# Use multilingual embedding model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Search in English, find Hindi/Marathi/Kannada conversations
```

### Performance at Scale

From Oracle documentation and testing:
- 1K memories: ~15ms search
- 10K memories: ~30ms search
- 100K memories: ~50ms search (with proper indexing)
- 1M+ memories: Sub-100ms (with partitioning)

Fast enough for real-time applications.

## Summary

### What We Built

1. Semantic memory storage using Oracle's native VECTOR type.  
2. Vector similarity search with cosine distance.  
3. Complete RAG pattern for context-aware AI responses.  
4. Production-ready foundation on enterprise database.  

### Why This Approach Works

1. Simplicity: One database, two operations (store, search).  
2. Data protection: Local embeddings - nothing leaves your server.  
3. Hybrid queries: Combine semantic search with SQL filters.  
4. Oracle handles complexity: Vector indexing, optimization done for you.  
5. Production-ready: ACID transactions, 99.95% SLA, enterprise support.  

### Real-World Impact

In the loan conversation agent I worked on, this would have:
- Eliminated manual transcript searching.
- Given agents context before calls.
- Improved borrower experience (no repeating information).
- Enabled "find similar cases" queries.
- Worked across multiple languages.

### Next Steps

To use this in your application:

1. Add your domain logic: User IDs, categories, metadata
2. Create vector indexes: For performance at scale
3. Connect real LLM: Replace simulated response with API
4. Implement memory management: Cleanup, consolidation
5. Add monitoring: Track search relevance, latency

### Resources

- [Blogpost Link](https://github.com/srinidhi-sat/agent-memory/blob/main/blogpost.md)
- [Requirements](https://github.com/srinidhi-sat/agent-memory/blob/main/requirements.txt)
- [README](https://github.com/srinidhi-sat/agent-memory/blob/main/README.md)
- [Oracle 26ai Vector Search Documentation](https://docs.oracle.com/en/database/oracle/oracle-database/26/vecse/)
- [Sentence Transformers](https://www.sbert.net/)
- [python-oracledb](https://python-oracledb.readthedocs.io/)
- [Python 3.9+](https://www.python.org/downloads/)
- [Oracle Always Free Tier](https://www.oracle.com/cloud/free/)

Questions or feedback?
Feel free to reach out or open an issue on GitHub.

*This project was built to explore semantic memory for AI agents, based on challenges encountered in a loan conversation system. The implementation demonstrates Oracle 26ai's vector capabilities with a practical, production-ready approach.*